# Тестирование работы API для тегирования видео

In [1]:
import pandas as pd
import numpy as np

## Загружаем тестовую выборку

In [2]:
x_test = pd.read_csv("x_test.csv", index_col=0).reset_index(drop=True)
x_test

,video_id,title,description
0,85c6ac4a4cfc3778cd2e65139d970dba,"Салют, Начальник | 4 серия",На очередной сельской планёрке обсуждают насущ...
1,7eecbd10bdaa596c2921ecb9bf7dbf69,Сколько Стоит Тачка: Галина Ржаксенская - от п...,"В этом выпуске вы узнаете, на чем ездит блогер..."
2,0d7b83b4d141158214acf42ab2ca0519,Какова Меркурия? | Выпуск 12,Женя вместе с астрологом попробуют разобраться...
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,Смехмашина | Выпуск 13,Подвели итоги розыгрыша. Поздравляем счастливо...
4,c093daa4f0e96d8f2e4736c9240898f3,Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышева,Героиней сегодняшнего выпуска Абьюз-шоу стала ...
...,...,...,...
205,3a84cf5e54a272f77470dd066f554edb,Yunan MotoTour в ГрандТуре «Байкальская миля 2...,Бурятский драматический театр в г. Улан-Удэ.
206,484b3600ebf13d5f11b1123b2fe4b142,Роман Юнусов и популярный озвучер Карен Арутюн...,В новом выпуске шоу «Спортивный Интерес» Рома ...
207,d70c45bc09cbbe583c9cede0477040a7,PykoJob | Выпуск 4 | Уютная гостиная с небольш...,Ведущая Алиса и дизайнер Наталья показывают ка...
208,2952b0a7148f5bb09247b22048cd11b9,"Команда ""3/21"" в ГрандТуре-2022: музей В.М. Шу...","""Счастье - оно обыкновенное. Каждый человек ка..."


## Обращаемся к API

Загрузка sample_submission

In [6]:
x_test.head(10)

,video_id,title,description
0,85c6ac4a4cfc3778cd2e65139d970dba,"Салют, Начальник | 4 серия",На очередной сельской планёрке обсуждают насущ...
1,7eecbd10bdaa596c2921ecb9bf7dbf69,Сколько Стоит Тачка: Галина Ржаксенская - от п...,"В этом выпуске вы узнаете, на чем ездит блогер..."
2,0d7b83b4d141158214acf42ab2ca0519,Какова Меркурия? | Выпуск 12,Женя вместе с астрологом попробуют разобраться...
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,Смехмашина | Выпуск 13,Подвели итоги розыгрыша. Поздравляем счастливо...
4,c093daa4f0e96d8f2e4736c9240898f3,Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышева,Героиней сегодняшнего выпуска Абьюз-шоу стала ...
5,e6cb1ed9cba95335b177b38d224a9cad,Домашняя кухня от шефа: Выпуск №15,Король кулинарии покажет рецепт короля всех су...
6,12ed5a6ab684e30de9f583c14effc2d9,"СarJitsu. 3 сезон, 2 серия. Ушу Мастер vs Макс...","CarJitsu — бои в формате POP MMA, где вместо р..."
7,e66b06cf902810f1d09e470947e91c80,Прогулка по стране - Нижний Новгород,"Сегодня нам предстоит прогулка по городу, веду..."
8,a189a686c63b0fbcbc9d07467e88be99,Смехмашина | Выпуск 7,"Очередной выпуск Смехмашины, в котором принима..."
9,7ae8dba100ff92da051677318f1bcb60,МАКСИМ НАРОДНЫЙ Выпуск №38 БОРЩ ИЛИ ЩИ? ВОТ В ...,Ведущий канала Максим предлагает своим подписч...


In [8]:
import requests

sample_submission = []
for i, data in x_test.head(10).iterrows():

    # with open(f'videos/85c6ac4a4cfc3778cd2e65139d970dba.mp4','rb') as video:
    tags = requests.post(
        url="http://10.8.0.6:8080/upload",
        # headers = {"Content-type": "multipart/form-data"},
        json=data.to_json(),
        # files={"file": video}
    )
    print(tags.text)
    sample_submission.append({
        "video_id": data["video_id"],
        "predicted_tags": tags.text,
    })

sample_submission = pd.DataFrame(sample_submission)
sample_submission

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u0430\u0440\u044c\u0435\u0440\u0430"
]

[
  "\u0421\u043f\u043e\u0440\u0442",
  "\u0422\u0440\u0430\u043d\u0441\u043f\u043e\u0440\u0442",
  "\u041a\u043

,video_id,predicted_tags
0,85c6ac4a4cfc3778cd2e65139d970dba,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
1,7eecbd10bdaa596c2921ecb9bf7dbf69,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
2,0d7b83b4d141158214acf42ab2ca0519,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
4,c093daa4f0e96d8f2e4736c9240898f3,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
5,e6cb1ed9cba95335b177b38d224a9cad,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
6,12ed5a6ab684e30de9f583c14effc2d9,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
7,e66b06cf902810f1d09e470947e91c80,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
8,a189a686c63b0fbcbc9d07467e88be99,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."
9,7ae8dba100ff92da051677318f1bcb60,"[\n ""\u0421\u043f\u043e\u0440\u0442"",\n ""\u0..."


In [3]:
sample_submission = pd.read_csv("../tagger/sample_submission.csv", index_col=0)
sample_submission

,video_id,predicted_tags
0,,
0,85c6ac4a4cfc3778cd2e65139d970dba,['Фильмы и анимация: Комедии']
1,7eecbd10bdaa596c2921ecb9bf7dbf69,['Личные финансы']
2,0d7b83b4d141158214acf42ab2ca0519,['Религия и духовность: Астрология']
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,['Массовая культура: Юмор и сатира']
4,c093daa4f0e96d8f2e4736c9240898f3,['Массовая культура: Беременности знаменитостей']
...,...,...
205,3a84cf5e54a272f77470dd066f554edb,['События и достопримечательности: Театральные...
206,484b3600ebf13d5f11b1123b2fe4b142,['Телевидение: Спортивные каналы']
207,d70c45bc09cbbe583c9cede0477040a7,['Дом и сад: Дизайн интерьера']


Загрузка ground_truth файла

In [4]:
ground_truth = pd.read_csv('y_test.csv', index_col=0)
ground_truth = ground_truth.join(x_test, how='inner')

Скрипт проверки

In [11]:
x_test.description[0], x_test.title[0], y_test.tag[0]

NameError: name 'y_test' is not defined

In [6]:
import pandas as pd
import argparse
import ast
import numpy as np

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission.copy()
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission.copy()
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"] in row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter


if __name__ == '__main__':

    try:
        pred_submission = sample_submission #pd.read_csv(pred_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке решения участника'
    try:
        true_submission = ground_truth #pd.read_csv(true_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке эталонного решения'


    final_score = find_iou_for_sample_submission(pred_submission, true_submission)
    print(final_score)



TypeError: Cannot perform 'or_' with a dtyped [object] array and scalar of type [bool]